In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(), # تحويل الصور إلى Tensor
    transforms.Normalize((0.1307,),(0.3081,))   # تطبيع البيانات
    ])

In [3]:
train_dataset = datasets.MNIST(root='./data',train=True,download=True,transform=transform)
test_dataset = datasets.MNIST(root='./data',train=False,download=True,transform=transform)

100%|██████████| 9.91M/9.91M [00:00<00:00, 17.6MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 478kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.42MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 15.2MB/s]


In [4]:
train_loader = DataLoader(train_dataset,batch_size=64,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=1000,shuffle=False)

In [23]:
class SimpleCNN(nn.Module):
  def __init__(self):
    super(SimpleCNN,self).__init__()
    self.conv1 = nn.Conv2d(in_channels=1,out_channels=32,kernel_size=3,stride=1,padding=1)
    self.relu1 = nn.ReLU()
    self.pool1 = nn.MaxPool2d(2,2)
    self.conv2d = nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,stride=1,padding=1)
    self.relu2 = nn.ReLU()
    self.pool2 = nn.MaxPool2d(2,2)
    self.fc1 = nn.Linear(in_features=7*7*64,out_features=128)
    self.relu3 = nn.ReLU()
    self.fc2 = nn.Linear(128,10)
  def forward(self,x):
    print("Input:",x.shape)
    x = self.conv1(x)
    print("After conv1:",x.shape)
    x = self.relu1(x)
    x = self.pool1(x)
    print("After poolq:",x.shape)
    x = self.conv2d(x)
    print("After conv2:",x.shape)
    x = self.relu2(x)
    x = self.pool2(x)
    print("After pool2:", x.shape)
    x = x.view(x.size(0),-1) # يسطّح الخرائط إلى شعاع بطول 64*7*7 لكل عينة
    print("After flatten:", x.shape)  # يطبع الشكل بعد التسطيح
    x = self.fc1(x)
    print("After fc1:", x.shape)
    self.relu3(x)
    x = self.fc2(x)
    print("After fc2 (logits):", x.shape)
    return x

In [24]:
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)

In [28]:
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    for epoch in range(epoch):
        running_loss = 0.0
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if batch_idx % 100 == 99:  # طباعة كل 100 دفعة
                print(f'[Epoch {epoch+1}, Batch {batch_idx+1}] loss: {running_loss / 100:.3f}')
                running_loss = 0.0
        print(f'انتهت الحقبة {epoch+1}')


In [30]:
device = torch.device(
"cuda" if torch.cuda.is_available() else "cpu" )
model.to(device)

SimpleCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=3136, out_features=128, bias=True)
  (relu3): ReLU()
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [31]:
train(model, device, train_loader, optimizer, criterion, epoch=1)

Streaming output truncated to the last 5000 lines.
After fc2 (logits): torch.Size([64, 10])
Input: torch.Size([64, 1, 28, 28])
After conv1: torch.Size([64, 32, 28, 28])
After poolq: torch.Size([64, 32, 14, 14])
After conv2: torch.Size([64, 64, 14, 14])
After pool2: torch.Size([64, 64, 7, 7])
After flatten: torch.Size([64, 3136])
After fc1: torch.Size([64, 128])
After fc2 (logits): torch.Size([64, 10])
Input: torch.Size([64, 1, 28, 28])
After conv1: torch.Size([64, 32, 28, 28])
After poolq: torch.Size([64, 32, 14, 14])
After conv2: torch.Size([64, 64, 14, 14])
After pool2: torch.Size([64, 64, 7, 7])
After flatten: torch.Size([64, 3136])
After fc1: torch.Size([64, 128])
After fc2 (logits): torch.Size([64, 10])
Input: torch.Size([64, 1, 28, 28])
After conv1: torch.Size([64, 32, 28, 28])
After poolq: torch.Size([64, 32, 14, 14])
After conv2: torch.Size([64, 64, 14, 14])
After pool2: torch.Size([64, 64, 7, 7])
After flatten: torch.Size([64, 3136])
After fc1: torch.Size([64, 128])
After fc2 

In [32]:
# --- دالة تقييم ---
def evaluate(model, device, test_loader):
    model.eval()  # وضع النموذج في طور التقييم
    correct = 0
    total = 0
    with torch.no_grad():  # إيقاف حساب التدرجات أثناء التقييم لتسريع العملية
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    print(f'Accuracy على بيانات الاختبار: {100 * correct / total:.2f}%')


In [34]:
# --- تنفيذ التدريب الكامل ---
epochs = 5
train(model, device, train_loader, optimizer, criterion, epochs)

# --- تقييم النموذج ---
evaluate(model, device, test_loader)

# --- حفظ النموذج ---
PATH = './mnist_cnn.pth'
torch.save(model.state_dict(), PATH)
print('💾 تم حفظ النموذج')

Streaming output truncated to the last 5000 lines.
After conv1: torch.Size([64, 32, 28, 28])
After poolq: torch.Size([64, 32, 14, 14])
After conv2: torch.Size([64, 64, 14, 14])
After pool2: torch.Size([64, 64, 7, 7])
After flatten: torch.Size([64, 3136])
After fc1: torch.Size([64, 128])
After fc2 (logits): torch.Size([64, 10])
Input: torch.Size([64, 1, 28, 28])
After conv1: torch.Size([64, 32, 28, 28])
After poolq: torch.Size([64, 32, 14, 14])
After conv2: torch.Size([64, 64, 14, 14])
After pool2: torch.Size([64, 64, 7, 7])
After flatten: torch.Size([64, 3136])
After fc1: torch.Size([64, 128])
After fc2 (logits): torch.Size([64, 10])
Input: torch.Size([64, 1, 28, 28])
After conv1: torch.Size([64, 32, 28, 28])
After poolq: torch.Size([64, 32, 14, 14])
After conv2: torch.Size([64, 64, 14, 14])
After pool2: torch.Size([64, 64, 7, 7])
After flatten: torch.Size([64, 3136])
After fc1: torch.Size([64, 128])
After fc2 (logits): torch.Size([64, 10])
Input: torch.Size([64, 1, 28, 28])
After conv